In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [2]:
with open('mxm_words.csv', 'r') as file:
    line = file.readline().strip()
words_list = line.split(',')
print(len(words_list))

5000


# MXM xref MuSe

In [3]:
mxmuse = pd.read_csv("mxmuse.csv", sep=";")
mxmuse.head

<bound method NDFrame.head of     Unnamed: 0                                        word_counts   
0            0  {6: 7, 7: 2, 24: 1, 38: 3, 42: 3, 47: 8, 77: 4...  \
1            1  {1: 32, 2: 16, 3: 20, 4: 21, 5: 3, 6: 5, 7: 22...   
2            2  {1: 10, 2: 8, 3: 1, 4: 8, 5: 7, 6: 3, 7: 2, 8:...   
3            3  {1: 17, 2: 17, 3: 15, 4: 26, 5: 8, 6: 19, 7: 4...   
4            4  {2: 11, 3: 2, 4: 6, 5: 2, 6: 8, 7: 3, 10: 6, 1...   
..         ...                                                ...   
81          81  {1: 12, 2: 16, 3: 14, 4: 24, 5: 8, 9: 12, 10: ...   
82          82  {1: 6, 2: 5, 3: 6, 4: 2, 5: 4, 6: 1, 7: 9, 8: ...   
83          83  {1: 2, 2: 8, 3: 2, 4: 5, 5: 5, 6: 5, 7: 4, 8: ...   
84          84  {1: 5, 2: 8, 3: 5, 4: 2, 5: 11, 6: 1, 7: 3, 8:...   
85          85  {1: 10, 2: 24, 3: 7, 4: 4, 5: 7, 6: 10, 7: 1, ...   

            Artist                           Title  valence_tags   
0      Soda Stereo                 Entre Caníbales      3.130000  \
1    

In [4]:
mxmuse.drop(columns=["Unnamed: 0"], inplace=True)
mxmuse

,word_counts,Artist,Title,valence_tags,arousal_tags,dominance_tags,seeds
0,"{6: 7, 7: 2, 24: 1, 38: 3, 42: 3, 47: 8, 77: 4...",Soda Stereo,Entre Caníbales,3.130000,2.997500,3.212500,['nostalgic']
1,"{1: 32, 2: 16, 3: 20, 4: 21, 5: 3, 6: 5, 7: 22...",Boyz II Men,On Bended Knee,6.472533,4.509067,5.800933,['romantic']
2,"{1: 10, 2: 8, 3: 1, 4: 8, 5: 7, 6: 3, 7: 2, 8:...",Pearl Jam,Inside Job,6.470000,3.962609,5.741304,['epic']
3,"{1: 17, 2: 17, 3: 15, 4: 26, 5: 8, 6: 19, 7: 4...",Busdriver,(Bloody Paw On The) Kill Floor,2.540000,2.045000,2.305000,['apocalyptic']
4,"{2: 11, 3: 2, 4: 6, 5: 2, 6: 8, 7: 3, 10: 6, 1...",Orbital,Illuminate,5.628889,3.875111,4.814222,['driving']
...,...,...,...,...,...,...,...
81,"{1: 12, 2: 16, 3: 14, 4: 24, 5: 8, 9: 12, 10: ...",22-20s,Hold On,3.033333,4.130000,4.336667,['brooding']
82,"{1: 6, 2: 5, 3: 6, 4: 2, 5: 4, 6: 1, 7: 9, 8: ...",Weezer,Mykel And Carli,6.586408,4.900583,5.632233,"['bittersweet', 'sweet']"
83,"{1: 2, 2: 8, 3: 2, 4: 5, 5: 5, 6: 5, 7: 4, 8: ...",Scott Matthews,City Headache,6.024516,3.301613,6.112258,['quiet']
84,"{1: 5, 2: 8, 3: 5, 4: 2, 5: 11, 6: 1, 7: 3, 8:...",K's Choice,20.000 Seconds,4.252000,3.878000,4.700000,['light']


### Create Bag of Words

In [5]:
bag_words = np.zeros((len(mxmuse), len(words_list)))
valence = []
arousal = []
dominance = []

for i, x in enumerate(mxmuse.values):
    word_count = eval(x[0])
    for word_index, count in word_count.items():
        bag_words[i, word_index] = count

print(bag_words)

[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0. 32. 16. ...  0.  0.  0.]
 [ 0. 10.  8. ...  0.  0.  0.]
 ...
 [ 0.  2.  8. ...  0.  0.  0.]
 [ 0.  5.  8. ...  0.  0.  0.]
 [ 0. 10. 24. ...  0.  0.  0.]]


In [6]:
v_train = (mxmuse['valence_tags'].values)[:69]
a_train = (mxmuse['arousal_tags'].values)[:69]
d_train = (mxmuse['dominance_tags'].values)[:69]

v_test = (mxmuse['valence_tags'].values)[69:]
a_test = (mxmuse['arousal_tags'].values)[69:]
d_test = (mxmuse['dominance_tags'].values)[69:]

In [7]:
# ~80/20 train/test split
bow_train = bag_words[:69]
bow_test = bag_words[69:]

In [24]:
v_model = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=0)
v_model.fit(bow_train, v_train)

RandomForestRegressor(max_depth=10, n_estimators=50, random_state=0)

In [25]:
a_model = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=0)
a_model.fit(bow_train, a_train)

RandomForestRegressor(max_depth=10, n_estimators=50, random_state=0)

In [26]:
d_model = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=0)
d_model.fit(bow_train, d_train)

RandomForestRegressor(max_depth=10, n_estimators=50, random_state=0)

In [48]:
v_pred = v_model.predict(bow_test)
a_pred = a_model.predict(bow_test)
d_pred = d_model.predict(bow_test)

v_pred = np.clip(v_pred, 0, 10)
a_pred = np.clip(a_pred, 0, 10)
d_pred = np.clip(d_pred, 0, 10)

v_mse = mean_squared_error(v_test, v_pred)
a_mse = mean_squared_error(a_test, a_pred)
d_mse = mean_squared_error(d_test, d_pred)

v_mae = mean_absolute_error(v_test, v_pred)
a_mae = mean_absolute_error(a_test, a_pred)
d_mae = mean_absolute_error(d_test, d_pred)

v_r2 = r2_score(v_test, v_pred)
a_r2 = r2_score(a_test, a_pred)
d_r2 = r2_score(d_test, d_pred)


print(f"Valence MSE: {v_mse:.5f}, MAE: {v_mae:.5f}, R2: {v_r2:.5f}")
print(f"Arousal MSE: {a_mse:.5f}, MAE: {a_mae:.5f}, R2: {a_r2:.5f}")
print(f"Dominance MSE: {d_mse:.5f}, MAE: {d_mae:.5f}, R2: {d_r2:.5f}")

Valence MSE: 2.52673, RMSE: 1.40352, R2: -0.28999
Arousal MSE: 1.25895, RMSE: 0.92452, R2: -0.38434
Dominance MSE: 1.15450, RMSE: 0.92377, R2: -0.19448


### Grid Search


#### Valence

In [36]:
param_grid = {
    'n_estimators': [17],
    'max_depth': [2],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    # 'max_features': [1.0, 'sqrt'],
    # 'bootstrap': [True, False],
    # 'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
}

model = RandomForestRegressor(random_state=0, )

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(bow_train, v_train)

best_params = grid_search.best_params_
best_v = grid_search.best_estimator_

In [37]:
best_params

{'max_depth': 2,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 17}

In [38]:
v_pred = best_v.predict(bow_test)
v_pred = np.clip(v_pred, 0, 10)

v_mse = mean_squared_error(v_test, v_pred)
v_mse

2.331021041360488

#### Arousal

In [39]:
param_grid = {
    'n_estimators': [50],
    'max_depth': [3],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    # 'max_features': [1.0, 'sqrt'],
    # 'bootstrap': [True, False],
    # 'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],

}

model = RandomForestRegressor(random_state=0)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(bow_train, a_train)

best_params = grid_search.best_params_
best_a = grid_search.best_estimator_

In [40]:
best_params

{'max_depth': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 50}

In [41]:
a_pred = best_a.predict(bow_test)
a_pred = np.clip(a_pred, 0, 10)

a_mse = mean_squared_error(a_test, a_pred)
a_mse

1.2280484557906886

#### Dominance

In [42]:
param_grid = {
    'n_estimators': [45],
    'max_depth': [5],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    # 'max_features': [1.0, 'sqrt'],
    # 'bootstrap': [True, False],
    # 'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],

}

model = RandomForestRegressor(random_state=0)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(bow_train, d_train)

best_params = grid_search.best_params_
best_d = grid_search.best_estimator_

In [43]:
best_params

{'max_depth': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 45}

In [44]:
d_pred = best_d.predict(bow_test)
d_pred = np.clip(d_pred, 0, 10)

d_mse = mean_squared_error(d_test, d_pred)
d_mse

1.1017151360635722

### Grid Search Results

In [46]:
v_mae = mean_absolute_error(v_test, v_pred)
a_mae = mean_absolute_error(a_test, a_pred)
d_mae = mean_absolute_error(d_test, d_pred)

print(f"Valence MSE: {v_mse:.5f}, MAE: {v_mae:.5f}")
print(f"Arousal MSE: {a_mse:.5f}, MAE: {a_mae:.5f}")
print(f"Dominance MSE: {d_mse:.5f}, MAE: {d_mae:.5f}")

Valence MSE: 2.33102, MAE: 1.34647
Arousal MSE: 1.22805, MAE: 0.94095
Dominance MSE: 1.10172, MAE: 0.91243
